In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def get_data_frame():
    current_folder = os.getcwd()
    file_path = os.path.join(current_folder,'..' ,'resource', 'RSMC_Best_Track_Data.csv')
    df = pd.read_csv(file_path)

    # ---------------------------

    direction_30_mapping = {'(symmetric circle)': 1, 'Northeast (NE)': 6, 'South (S)': 3,
                            'East (E)': 5, 'Southeast (SE)': 9, 'West (W)': 4, 'North (N)': 2,
                            'Northwest (NW)': 6, 'Southwest (SW)': 8}
    df['Direction of the longest radius of 30kt winds or greater'] = df[
        'Direction of the longest radius of 30kt winds or greater'].map(direction_30_mapping)

    direction_50_mapping = {'(symmetric circle)': 1, 'Northeast (NE)': 6, 'South (S)': 3,
                            'East (E)': 5, 'Southeast (SE)': 9, 'West (W)': 4, 'North (N)': 2,
                            'Northwest (NW)': 6, 'Southwest (SW)': 8,
                            'No direction (Longest radius of 50kt winds is 0)': 9}
    df['Direction of the longest radius of 50kt winds or greater'] = df[
        'Direction of the longest radius of 50kt winds or greater'].map(direction_50_mapping)

    indicator_mapping = {'#': 1, ' ': 0}
    df['Indicator of landfall or passage'] = df['Indicator of landfall or passage'].map(indicator_mapping)

    # ----------------------

    df = df.drop(['International number ID', 'Name of the storm', 'Grade'], axis=1)
    columns_to_check = [
        'Direction of the longest radius of 50kt winds or greater',
        'The longest radius of 50kt winds or greater',
        'The shortest radius of 50kt winds or greater',
        'Direction of the longest radius of 30kt winds or greater',
        'The longest radius of 30kt winds or greater',
        'The shortest radius of 30kt winds or greater'
    ]
    df = df.dropna(subset=columns_to_check)

    # -----------------------

    one_hot_encoded = pd.get_dummies(df['Direction of the longest radius of 50kt winds or greater'],
                                     prefix='Direction of the longest radius of 50kt winds or greater', dtype=int)
    df = pd.concat([df, one_hot_encoded], axis=1)

    one_hot_encoded2 = pd.get_dummies(df['Direction of the longest radius of 30kt winds or greater'],
                                      prefix='Direction of the longest radius of 30kt winds or greater', dtype=int)
    df = pd.concat([df, one_hot_encoded2], axis=1)
    df = df.drop(['Direction of the longest radius of 50kt winds or greater',
                  'Direction of the longest radius of 30kt winds or greater'], axis=1)

    # ----------------------

    df['Time of analysis'] = pd.to_datetime(df['Time of analysis'])
    df['Time of analysis'] = df['Time of analysis'].apply(lambda x: x.timestamp())

    # -----------------------
    return df

In [3]:
df = get_data_frame()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27299 entries, 25272 to 68731
Data columns (total 26 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Time of analysis                                              27299 non-null  float64
 1   Latitude of the center                                        27299 non-null  float64
 2   Longitude of the center                                       27299 non-null  float64
 3   Central pressure                                              27299 non-null  int64  
 4   Maximum sustained wind speed                                  27299 non-null  float64
 5   The longest radius of 50kt winds or greater                   27299 non-null  float64
 6   The shortest radius of 50kt winds or greater                  27299 non-null  float64
 7   The longest radius of 30kt winds or greater                   27299 

In [4]:
max_lat = df['Latitude of the center'].max()
min_lat = df['Latitude of the center'].min()


max_long = df['Longitude of the center'].max()
min_long = df['Longitude of the center'].min()

print(f"max lat: {max_lat}")
print(f"min lat: {min_lat}")
print(f"max long: {max_long}")
print(f"min long: {min_long}")

max lat: 50.0
min lat: 1.5
max long: 182.3
min long: 97.5


In [5]:
def min_max_denormalize(normalized_data, min_value, max_value):
    denormalized_data =  normalized_data * (max_value - min_value) + min_value
    return denormalized_data

In [6]:
lat = min_max_denormalize(0.478774, 97.5, 182.3)
print(lat)

138.1000352
